In [1]:
!pip install topmost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 50.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 83.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [2]:
import numpy as np
from tqdm import tqdm
import scipy.sparse
from topmost.preprocessing import Preprocessing
import topmost.preprocessing.preprocessing as prepro
import importlib
from topmost.utils.logger import Logger
import fasttext
import fasttext.util
import nltk
from nltk.corpus import stopwords

In [3]:
logger = Logger("WARNING")
logger = Logger("DEBUG")
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))
fasttext.util.download_model('es', if_exists='ignore')  # Español

# print(stopwords_es)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'cc.es.300.bin'

In [4]:
def make_word_embeddings_es(vocab):
    # Cargar el modelo binario preentrenado
    fasttext_model = fasttext.load_model("cc.es.300.bin")

    # glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
    word_embeddings = np.zeros((len(vocab), fasttext_model.get_dimension()))

    num_found = 0

    for i, word in enumerate(tqdm(vocab, desc="loading SPANISH 🍀 word embeddings")):
        word_embeddings[i] = fasttext_model.get_word_vector(word)
        num_found += 1

    logger.info(f'number of found embeddings: {num_found}/{len(vocab)}')

    return scipy.sparse.csr_matrix(word_embeddings)

prepro.make_word_embeddings = make_word_embeddings_es

In [5]:
jsonlist_path = '/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed'
output_path = './datasets/CMed'
    
# os.makedirs(jsonlist_path, exist_ok=True)
# os.makedirs(output_path, exist_ok=True)

preprocessing = Preprocessing(min_doc_count=10, stopwords=stopwords_es)

rst = preprocessing.preprocess_jsonlist(dataset_dir=jsonlist_path, label_name="label")

preprocessing.save(output_path, **rst)

2025-01-12 08:46:26,676 - TopMost - Found training documents 1208 testing documents 658
2025-01-12 08:46:26,679 - TopMost - label2id: {'carta-al-editor': 0, 'ciencias-basicas-biomedicas': 1, 'ciencias-clinicas-y-patologicas': 2, 'ciencias-de-la-educacion': 3, 'ciencias-epidemiologicas-y-salubristas': 4, 'ciencias-quirurgicas': 5, 'ciencias-sociales': 6, 'ciencias-tecnologicas': 7, 'editorial': 8, 'estudios-bibliometricos-y-cienciometricos': 9, 'historico': 10, 'nefrologia-al-dia': 11, 'reflexiones': 12, 'semblanza': 13}
parsing texts: 100%|██████████| 1208/1208 [00:03<00:00, 375.99it/s]
2025-01-12 08:46:45,998 - TopMost - Real vocab size: 19384
2025-01-12 08:46:46,018 - TopMost - Real training size: 1208 	 avg length: 1335.378
parsing texts: 100%|██████████| 658/658 [00:01<00:00, 377.12it/s]
2025-01-12 08:46:48,739 - TopMost - Real testing size: 658 	 avg length: 1347.125
loading SPANISH 🍀 word embeddings: 100%|██████████| 19384/19384 [00:00<00:00, 85854.12it/s]
2025-01-12 08:46:57,070